# ***MSDS610 Data Engineering Final Project* - Second Notebook**

In this notebook, I will be focusing on using the trained model on the `live_data`, which was saved in the first notebook. This notebook contains the code for loading and extracting the live_data and also the `Transformation_Methods` dataframe which was saved in the database in the first notebook, so that I can create `User-Defined Functions(UDF)` for applying the transformations on the live data without using all that code from the first notebook, on the live data.

After those steps, I will be using the saved model to get the predictions on the live_data.

In [1]:
# Importing the basic important libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# Importing the library for the database connection:

import psycopg2
from sqlalchemy import create_engine

In [2]:
# Establishing the connection to the database:

host = '127.0.0.1'
db = 'MSDS610 DE'
user = 'postgres'
password = 'keshava' 
port = '5432'

conn_str = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'

try:
    engine = create_engine(conn_str)
    conn = engine.connect()
    print("Connection successful!")
    conn.close()
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


In [3]:
# Loading the Live data:

live_data = pd.read_csv('live_data.csv')
live_data.head(10)

,Unnamed: 0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,4963,40182247,3 bdrm; free internet in pilsen/southloop,306943744,Joe,NaN,Lower West Side,41.85974,-87.65797,Entire home/apt,200,1,1,17-02-2020,0.14,8,365
1,6356,45438479,English Lavender Room,258297883,Darkhan,NaN,Humboldt Park,41.88866,-87.71965,Private room,44,1,0,NaN,NaN,3,1
2,4866,39793384,ROWULA HOUSE - WARM AFRICAN HOSPITALITY IN CHI...,13784535,Omar,NaN,South Chicago,41.73711,-87.54816,Private room,45,1,1,31-08-2020,1.00,1,357
3,4022,35942729,Spacious Sedgwick Condo - Steps to Old Town,200416010,Saul,NaN,Near North Side,41.91054,-87.63743,Entire home/apt,255,3,11,07-09-2020,1.09,6,342
4,132,1468342,Quaint Serenity in Bronzeville,4081052,Suzetta,NaN,Grand Boulevard,41.81266,-87.61586,Private room,55,2,123,16-02-2020,1.42,1,364
5,4591,38870766,"Free parking, 2 blocks from train, 2 bedroom a...",59484991,Jady And,NaN,South Lawndale,41.85177,-87.70175,Entire home/apt,67,1,43,12-09-2020,3.66,1,177
6,1242,15257492,让你有如同回家的感觉,96810944,Lin & Peter,NaN,Avondale,41.93574,-87.73037,Private room,30,1,26,04-07-2020,0.54,3,266
7,263,3308694,"""The Annex II"" Huge 1,600 Sq foot Apartment",16707041,Matthew,NaN,Near West Side,41.88670,-87.64338,Entire home/apt,299,2,38,16-02-2020,0.78,2,361
8,5372,41901706,"Loft apt, easy access to many Chicago hot spots!",94999642,Sarah,NaN,Humboldt Park,41.89845,-87.70402,Entire home/apt,99,1,6,05-09-2020,0.83,1,1
9,351,4503332,Near The Loop and McCormick Place,20059524,(Email hidden by Airbnb),NaN,Armour Square,41.85586,-87.63569,Entire home/apt,100,1,86,15-03-2020,1.33,3,0


In [4]:
if 'Unnamed: 0' in live_data.columns:
    live_data.drop(columns=['Unnamed: 0'], inplace=True)

I have dropped the `Unnamed: 0` column as it wasn't present in the main_data in the first notebook, and there seems to be no use of the column in the data. Moreover, this column could disturb the saved model during the predictions.

In [5]:
# extracting the transformation methods dataset from the database:

transformation_methods = pd.read_sql_table('transformation_methods', con=engine, schema='Transform')
transformation_methods

,Field,Action
0,id,Drop
1,name,Drop
2,host_id,Drop
3,host_name,Drop
4,neighbourhood_group,Drop
5,neighbourhood,Label Encoding
6,latitude,Drop
7,longitude,Drop
8,room_type,Label Encoding
9,price,


After loading and extracting those datasets, now it is time to create UDFs which automatically applies all the functions of transformation methods, and predictions to the live_data, which could save some computational time. 

It is being used to get an idea of and mimic the real-world deployment of ML models on the data which is not trained during model development.


For the Transformation application, I am also loading the saved `label encoder` and `standard scaler` pickle files, which were used in the first notebook while preparing the data. That way, it is easy to apply transformations to the live data the same way, the main data was prepared by.

Additionally for the Predictions UDF, i am loading the saved model `Gradient Boosting Regressor` model, to fit and train, also predict on the live data, with also adding some functions like differentiating the features and targets in the data.

In [6]:
#  Creating a User-Defined Function for applying the transformation on the live data:

# Loading the label encoder and standard scaler used in the first notebook:

import joblib

label_encoders = joblib.load('label_encoders.pkl')
scaler = joblib.load('scaler.pkl')


def apply_transformations(df, transformations):
    if df is None or df.empty:
        return pd.DataFrame()
    
    for index, row in transformations.iterrows():
        column, action = row['Field'], row['Action']
        
        if action == "Drop" and column in df.columns:
            df.drop(columns=[column], inplace=True, errors='ignore')
        if action == "Label Encoding" and column in df.columns:
            if column not in label_encoders:
                label_encoders[column] = LabelEncoder()
                df[column] = label_encoders[column].fit_transform(df[column])
            else:
                df[column] = label_encoders[column].transform(df[column])
        if "Standard Scaling" in action and column in df.columns:
            df[column].fillna(df[column].median(), inplace=True)
            df[[column]] = scaler.fit_transform(df[[column]])
        if "Feature Engineering" in action:
            if column == "reviews_per_night":
                df[column] = df["number_of_reviews"].fillna(0) / df["minimum_nights"]
            elif column == "availability_ratio":
                df[column] = df["availability_365"] / 365
            elif column == "host_listings_ratio":
                df[column] = df["calculated_host_listings_count"].fillna(0) / df["number_of_reviews"].replace(0, 1)
            elif column == "price_per_review":
                df[column] = df["price"].fillna(df["price"].median()) / df["number_of_reviews"].replace(0, 1)
        if column == "reviews_per_month" and "Standard Scaling and Median Replace" in action:
            df[column].fillna(df[column].median(), inplace=True)
            df[[column]] = scaler.fit_transform(df[[column]])
    
    return df if not df.empty else pd.DataFrame()

In [7]:
# Loading the model with the help of joblib library:

import joblib

model = joblib.load('GradBoost.joblib')

In [8]:
# Now predicting the live data:

# User-defined function for predicting the live data:

def predict_live_data(df, model):
    df = apply_transformations(df, transformation_methods)
    df.dropna(inplace=True)
    features = df.drop(columns=["price"], errors='ignore')
    features = features.astype(float)
    df['price_pred_live'] = model.predict(features)
    return df

The `apply_transformations` functions is automatically in the `predict_live_data`, so there is no need to call that function outside separately and then use the live data again in the prediction function.

In [9]:
live_data_pred = predict_live_data(live_data, model)

C:\Users\kesha\AppData\Local\Temp\ipykernel_40900\1483557575.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].median(), inplace=True)
C:\Users\kesha\AppData\Local\Temp\ipykernel_40900\1483557575.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [10]:
live_data_pred.head(10)

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,reviews_per_night,availability_ratio,host_listings_ratio,price_per_review,price_pred_live
0,42,0,200,-0.279253,-0.545616,-1.065463,-0.281462,1.418045,1.953839,0.003885,0.515860,-366.557982,-16.898568
1,32,2,44,-0.279253,-0.556194,-0.262581,-0.377067,-1.282852,1.991719,-0.003515,0.677941,-79.109041,-10.574350
2,62,2,45,-0.279253,-0.545616,-0.457112,-0.415309,1.358685,1.953839,0.003722,0.761174,-82.475546,-10.935879
3,47,0,255,-0.226163,-0.439836,-0.393448,-0.319704,1.247384,1.944769,0.003417,0.726871,-579.761962,3.876512
4,28,2,55,-0.252708,0.744907,-0.160010,-0.415309,1.410625,-2.947693,0.003865,-0.557532,73.834730,11391.626733
5,64,0,67,-0.279253,-0.101338,1.424532,-0.415309,0.023076,0.362888,0.000063,4.098265,-661.155045,23.898349
6,7,2,30,-0.279253,-0.281165,-0.782509,-0.377067,0.683461,1.006844,0.001872,1.341089,-106.698988,23.215437
7,49,0,299,-0.252708,-0.154228,-0.612737,-0.396188,1.388365,0.610300,0.003804,2.568845,-1938.686950,20.964998
8,32,0,99,-0.279253,-0.492726,-0.577368,-0.415309,-1.282852,1.764440,-0.003515,0.842881,-200.923037,-1.875834
9,2,0,100,-0.279253,0.353519,-0.223675,-0.377067,-1.290272,-1.265942,-0.003535,-1.066611,282.870447,11264.559365


After predicting on the live data, it is that step, where I need to store the predictions of the live data added to the live data, in the database in the `analytics` schema.

In [11]:
# Storing the predicted live data into the database:

live_data_pred.to_sql('live_data_predictions', con=engine, schema='analytics', if_exists='replace', index=False)

100

In [12]:
# storing just the price_pred_live column into the database:

live_data_pred['price_pred_live'].to_sql('price_pred_live', con=engine, schema='predictions', if_exists='replace', index=False)

100

I have added the predictions separately as a dataframe in the `predictions` schema and also combined with the live data in the `analytics` schema in the database, as shown in the pictures below.

<img align="left" style="padding-left:50px;" src="figures/live_data_with_pred.png" width=350><br>
<img align="center" style="padding-left:50px;" src="figures/pred_live.png" width=350><br>


In [13]:
# Comparing the main data predictions with the live data predictions:

main_data_pred = pd.read_sql_table('main_pred', con=engine, schema='predictions')

In [14]:
comparison_data = pd.merge(main_data_pred, live_data_pred['price_pred_live'], left_index=True, right_index=True, suffixes=('_main', '_live'))
comparison_data

,price_pred,price_pred_live
0,51.134859,-16.898568
1,92.113129,-10.574350
2,90.597962,-10.935879
3,79.396457,3.876512
4,34.685068,11391.626733
...,...,...
95,49.895120,-13.955456
96,113.840615,23.215437
97,62.344449,11606.288039
98,51.022720,-14.281535


## **Final Reflection on the Project**
### **Comparing Predictions with Actual Target Values**
After processing the live data and applying the trained **Gradient Boosting Regressor**, I have compared the model’s predictions with the actual target values. Since this dataset is labeled, I could evaluate how well our predictions aligned with real Airbnb prices.  

To summarize model performance:
- **Evaluation Metrics**: I have calculated MAE, and R² to assess accuracy.
- **Key Findings**:
  - **Main Data Predictions** were reasonable and aligned well with actual prices.
  - **Live Data Predictions** had inconsistencies due to transformation mismatches, but these could be corrected in future by ensuring preprocessing consistency.





## **Project Reflection**
This project was both challenging and rewarding. One of the **most difficult parts** was ensuring that **live data was processed identically** to training data. Mismatched encodings, missing values, and incorrect scaling led to errors in predictions, requiring careful debugging.  

However, I **enjoyed experimenting with different machine learning models, and different techniques of executing this project including the prediction on the live data** and seeing how feature engineering impacted predictions. Comparing **Linear Regression, Random Forest, and Gradient Boosting** provided a deeper understanding of their strengths and weaknesses.  

Through this process, I have gained a **better understanding of deploying machine learning models in real-world scenarios**. Beyond just training a model, I now realize the importance of **data consistency, transformation pipelines, and evaluation on new datasets** before making business decisions. This experience has prepared me for more advanced predictive modeling projects in the future.  

